In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!dir
!python -m pip install "dask[complete]"
!python -m pip install pynndescent
!python -m pip install tqdm
!python -m pip install umap

drive  sample_data
     |████████████████████████████████| 125 kB 5.3 MB/s 
     |████████████████████████████████| 791 kB 43.6 MB/s 
     |████████████████████████████████| 786 kB 69.4 MB/s 
     |████████████████████████████████| 779 kB 59.8 MB/s 
     |████████████████████████████████| 778 kB 57.1 MB/s 
     |████████████████████████████████| 776 kB 73.2 MB/s 
     |████████████████████████████████| 769 kB 71.0 MB/s 
     |████████████████████████████████| 766 kB 29.2 MB/s 
     |████████████████████████████████| 1.0 MB 64.6 MB/s 
     |████████████████████████████████| 722 kB 61.2 MB/s 
     |████████████████████████████████| 722 kB 75.4 MB/s 
     |████████████████████████████████| 715 kB 78.3 MB/s 
     |████████████████████████████████| 705 kB 72.0 MB/s 
     |████████████████████████████████| 699 kB 81.3 MB/s 
     |████████████████████████████████| 696 kB 78.7 MB/s 
     |████████████████████████████████| 684 kB 76.4 MB/s 
     |████████████████████████████████| 679 kB 58.5 MB

In [ ]:
# basic imports 
import os
import shutil
import numpy as np
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm
import keras

# plotting
import matplotlib.pyplot as plt

# improving plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('bmh')

## Custom utilities

I build some custom utilities to isolate some core functionalities of my code. I'll explain them in the post, but if you want to dig deeper please refer to [this script]() on the repository.

In [ ]:
# basic imports 
import os
import shutil
import numpy as np
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm

# plotting
import matplotlib.pyplot as plt

# improving plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('bmh')

# using a pre-trained net
#from tensorflow.keras.applications.xception import preprocess_input
#from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input
#from tensorflow.keras.applications.inception_v3 import preprocess_input

from tensorflow.keras.preprocessing import image

# nearest neighbors
from sklearn.neighbors import KDTree
from pynndescent import NNDescent

# tools for creating embedding plots
from sklearn.cluster import KMeans
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


# function to load data
def build_metadata():

    # list for meta_df
    meta_df = []

    # traversing folders and images
    for base_path, breed_folder, imgs in tqdm(os.walk('/content/drive/MyDrive/CS465/project/Images')):
        for img in imgs:

            # gathering metadata
            pet_id = f'{base_path}/{img}'
            breed = '-'.join(base_path.split('/')[-1].split('-')[1:])

            # dataframe with this info
            temp_df = pd.DataFrame({'breed': breed}, index=pd.Index([pet_id], name='pet_id'))
            meta_df.append(temp_df)

    # returning full dataframe
    meta_df = pd.concat(meta_df)
    return meta_df

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# function to read all images into array
def read_images(pet_ids, target_width=244, target_height=244):
    
    # list with images and ids
    images = []
    processed_ids = []
    
    # loop for each pet id in the main dataframe
    for pet_id in tqdm(pet_ids):
        
        try:
            
            # reading image and putting it into machine format
            img = image.load_img(pet_id, target_size=(target_width, target_height))
            img = image.img_to_array(img)
            img = preprocess_input(img)
            
            # saving
            images.append(img)
            processed_ids.append(pet_id)
        
        # do nothing if passes
        except:
            pass
        
    return np.array(images), np.array(processed_ids)

# function to extract and save features from images
def extract_features(pet_ids, extractor):
    
    # getting features iterating
    features_df = pd.DataFrame()
    for pet_chunk in chunks(pet_ids, 2048):
  
        # reading and processing images
        images, processed_ids = read_images(pet_chunk)
        result = extractor.predict(images, batch_size=128, verbose=1, use_multiprocessing=True)
        result = pd.DataFrame(result, index = pd.Index(processed_ids, name='pet_id'))
        features_df = pd.concat([features_df, result])

    # saving df
    return features_df

# report of zeca's comparables
def get_prototypes_report(path, index, extractor, transform_fn, meta_df, features_df, k=50):
    
    # features from zeca
    features = extract_features([path], extractor)

    # querying zeca NNs
    nns = index.query(transform_fn(features), k=k)
    
    # breeds of comparable dogs
    comps_breed = meta_df.iloc[nns[0][0]]['breed']
    breed_counts = comps_breed.value_counts()
    print('Most Frequent Breeds:')
    print((breed_counts/breed_counts.sum()).head(10))
    
    # comps
    comps_fig_path = features_df.index[nns[0][0]].values

    # opening matplotlib figure
    fig = plt.figure(figsize=(20, 10), dpi=100)

    # loop for all figures
    for i, path in enumerate(comps_fig_path):
        plt.subplot(5, 10, i+1)
        plt.imshow(plt.imread(path))
        plt.title(comps_breed.iloc[i], fontsize=9)
        plt.grid(b=None)
        plt.xticks([]); plt.yticks([])
    
def plot_dog_atlas(embed, meta_df, title, ax):

    # fitting kmeans to get evenly spaced points on MAP
    km = KMeans(n_clusters=100)
    km.fit(embed)

    # getting these centroids
    centroids = km.cluster_centers_
    medoids = (
        pd.DataFrame(embed)
        .apply(lambda x: km.score(x.values.reshape(1,-1)), axis=1)
        .groupby(km.predict(embed))
        .idxmax()
    )

    # images to plot
    img_to_plot = meta_df.index.values[medoids]
    
    # plotting a light scatter plot
    #fig, ax = plt.subplots(figsize=(12,6), dpi=120)
    ax.scatter(embed[:,0], embed[:,1], s=2, alpha=0.1, color='black')

    # loop adding pictures to plot
    for i, img in enumerate(img_to_plot):

        img = plt.imread(img)
        imagebox = OffsetImage(img, zoom=0.1)
        imagebox.image.axes = ax

        ab = AnnotationBbox(imagebox, embed[medoids[i]], pad=0)
        ax.add_artist(ab)
        
    # title and other info
    ax.set_title(title)
    ax.set_xlabel('first UMAP dimension')
    ax.set_ylabel('second UMAP dimension')
    plt.grid(b=None)
    ax.set_xticks([]); ax.set_yticks([])
        
def plot_embedding(embed, zeca_embed, title, colors):
    
    # opening figure
    #fig, ax = plt.subplots(figsize=(12,6), dpi=120)
    
    # running scatterplot for all dogs and zeca
    plt.scatter(embed[:,0], embed[:,1], s=2, c=colors, cmap='gist_rainbow')
    plt.scatter(zeca_embed[:,0], zeca_embed[:,1], s=300, c='black', label='Zeca', marker='*')
    
    # title and other info
    plt.title(title)
    plt.xlabel('first UMAP dimension')
    plt.ylabel('second UMAP dimension')
    plt.xticks([]); plt.yticks([])
    plt.grid(b=None)
    plt.legend()

## Data

The data is divided into 120 folders, each representing a breed, that contain several dog pictures each. The `build_metadata` function builds a simple dataframe which contains a single column `breed` and the path to the corresponding image as index.

In [ ]:
# reading data
meta_df = build_metadata()
meta_df.head()

121it [00:42,  2.86it/s]


,breed
pet_id,
/content/drive/MyDrive/CS465/project/Images/n02113186-Cardigan/n02113186_1030.jpg,Cardigan
/content/drive/MyDrive/CS465/project/Images/n02113186-Cardigan/n02113186_10816.jpg,Cardigan
/content/drive/MyDrive/CS465/project/Images/n02113186-Cardigan/n02113186_10077.jpg,Cardigan
/content/drive/MyDrive/CS465/project/Images/n02113186-Cardigan/n02113186_10505.jpg,Cardigan
/content/drive/MyDrive/CS465/project/Images/n02113186-Cardigan/n02113186_10535.jpg,Cardigan


As expected, we have 120 breeds. Also, we have 20580 images, as a I joined the train and test sets of the original dataset, as I need the most data I can get.

In [ ]:
# number of unique breeds after filter
print('number of unique breeds:', meta_df['breed'].nunique())
print('number of rows in the dataframe:', meta_df['breed'].shape[0])

number of unique breeds: 120
number of rows in the dataframe: 20580


We reserve the images' paths for use later:

In [ ]:
# creating list with paths
paths = meta_df.index.values

## Feature extraction 

The first step is extracting features from the images using a pretrained neural network. I chose `Xception` based on its good results on this [Kaggle Kernel](https://www.kaggle.com/gaborfodor/dog-breed-pretrained-keras-models-lb-0-3/#data), and for it being relatively lightweight for quick inference.

In [ ]:
# using a pre-trained net
# Xception Model
from tensorflow.keras.applications.xception import Xception
# VGG19 Model
from tensorflow.keras.applications.vgg19 import VGG19
# DenseNet201 Model
from tensorflow.keras.applications.densenet import DenseNet201
# InceptionV3 Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.preprocessing import image

# instance of feature extractor
# extractor = VGG19(include_top=False, pooling='avg')
# extractor = VGG19(include_top=False, pooling='max')
# extractor = Xception(include_top=False, pooling='avg')
# extractor = Xception(include_top=False, pooling='max')
extractor = DenseNet201(include_top=False, pooling='none')
# extractor = DenseNet201(include_top=False, pooling='avg')
# extractor = DenseNet201(include_top=False, pooling='max')
# extractor = InceptionV3(include_top=False, pooling='avg')
# extractor = InceptionV3(include_top=False, pooling='max')

74850304/74836368 [==============================] - 1s 0us/step


Before doing the feature extrating we need to do make use of transfer learning and fine tuning to improve the prediciton qulaity of our model

In [ ]:
# Source: https://www.tensorflow.org/guide/keras/transfer_learning
# freeze the base extractor model
extractor.trainable = False

# create new model to put on top of the base extractor model
inputs = keras.Input(shape=(244, 244, 3))
x = extractor(inputs, training=False)

# Source: Assignment 3
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, Conv1D, MaxPooling2D

# x = Conv2D(32, kernel_size=(3, 3), activation='softmax')(x)
# x = Conv2D(64, kernel_size=(3, 3), activation='softmax')(x)
# # (1) First batch normalization layer
# x = keras.layers.BatchNormalization()(x)
# # (2) Two Convolution Layers
# # doubleing the number of output filters each convolution
# x = Conv2D(128, (3, 3), activation='softmax')(x)
# x = Conv2D(256, (3, 3), activation='softmax')(x)
# # (1) Second batch normalization layer
# x = keras.layers.BatchNormalization()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(meta_df['breed'].nunique(), activation='softmax')(x)
x = Dropout(0.5)(x)

# A Dense classifier with 120 units (120 classification)
outputs = keras.layers.Dense(meta_df['breed'].nunique())(x)
extractor = keras.Model(inputs, outputs)
extractor.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 244, 244, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 7, 7, 120)         18552504  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 120)         0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 120)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               129720    
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0   

#Gather the Dataset

In [ ]:
import tensorflow.data.experimental
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    "/content/drive/MyDrive/CS465/project/Images", labels='inferred', label_mode='categorical',
    class_names=None, color_mode='rgb', batch_size=32, image_size=(244, 244),
    shuffle=True, seed=100, validation_split=.1, subset='training',
    interpolation='bilinear', crop_to_aspect_ratio=True
)

print("Number of training samples: %d" % tensorflow.data.experimental.cardinality(train_ds))
#print(
#    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
#)

Found 20580 files belonging to 120 classes.
Using 18522 files for training.
Number of training samples: 579


In [ ]:
########################################
# get entire image dataset
length = len(paths)
mid_index = length//10
first_nth = paths[:mid_index]
dataset = read_images(first_nth)
images, ids = dataset[0], dataset[1]

# get entire image dataset
#dataset, ids = read_images(paths)
########################################

100%|██████████| 2058/2058 [08:05<00:00,  4.24it/s]


#Train the custom top model

In [ ]:
# Train the new model
from tensorflow.keras.optimizers import Adam
extractor.compile(optimizer=Adam(), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

extractor.fit(train_ds, batch_size=128, epochs=10, verbose=1,shuffle=True, use_multiprocessing=True)# validation_split=0.2, 

Epoch 1/10
579/579 [==============================] - 1114s 2s/step - loss: 4.7899 - categorical_accuracy: 0.0092
Epoch 2/10
579/579 [==============================] - 797s 1s/step - loss: 4.7844 - categorical_accuracy: 0.0119
Epoch 3/10
579/579 [==============================] - 799s 1s/step - loss: 4.7829 - categorical_accuracy: 0.0112
Epoch 4/10
579/579 [==============================] - 972s 2s/step - loss: 4.7821 - categorical_accuracy: 0.0110
Epoch 5/10
579/579 [==============================] - 982s 2s/step - loss: 4.7817 - categorical_accuracy: 0.0111
Epoch 6/10
579/579 [==============================] - 930s 2s/step - loss: 4.7820 - categorical_accuracy: 0.0113
Epoch 7/10
579/579 [==============================] - 969s 2s/step - loss: 4.7807 - categorical_accuracy: 0.0105
Epoch 8/10
579/579 [==============================] - 825s 1s/step - loss: 4.7799 - categorical_accuracy: 0.0125
Epoch 9/10
579/579 [==============================] - 879s 2s/step - loss: 4.7772 - categorical

The function `extract_features` gets a list of paths, an `extractor` (the Xception net in this case), and returns a dataframe with features. We save the dataframe so we don't need to run the process all the time (it takes ~15 minutes on my machine).

In [ ]:
# if we havent extracted features, do it
if not os.path.exists('/content/drive/MyDrive/CS465/project/transferLearning_densenet201_avgPooling_features.csv'):
    features_df = extract_features(paths, extractor)
    features_df.to_csv('/content/drive/MyDrive/CS465/project/transferLearning_densenet201_avgPooling_features.csv')
    
# read features
features_df = pd.read_csv('/content/drive/MyDrive/CS465/project/transferLearning_densenet201_avgPooling_features.csv', index_col='pet_id')

As the extraction pipeline can't process some of the images, we need to realign our metadata index with the extraction's index, so they have the same images, in the same order:

In [ ]:
# realign index with main df
meta_df = meta_df.loc[features_df.index]

## Modeling

Now we can start modeling. We'll build a Logistic Regression to classify breeds on top of the Xception's features, and apply this model on a picture of Zeca. However, for the sake of explainability, we'll also create a nearest-neighbors model, so we can supply prototypes, comparable dogs to Zeca that can support the model's predictions.

Let's start with data preparation!

### Data preparation

Just explicitly splitting our design matrix `X` and target variable `y` into train and test sets (90%/10% split, stratified). We encode `y` using `LabelEncoder` as this is a multiclass classification problem.

In [ ]:
# label encoder for target and splitter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# defining design matrix
X = features_df.copy().values

# defining target
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(meta_df['breed'])

# splitting train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

### Dimensionality reduction with PCA

We run PCA in the Xception's features. We have two reasons for that:

1. **Efficiency.** PCA can retain 96% of variance with half the features (1024 instead of 2048). This helps everything run faster further in the pipeline.
2. **Whitening.** Whitening is the PCA's capability of returning a matrix where features have mean 0, variance 1, and are uncorrelated. This will be important as it allows us to interpret the  Logistic Regression coefficients as feature importances.

We fit PCA with the following code:

In [ ]:
# PCA
from sklearn.decomposition import PCA

# instance of PCA
# pca = PCA(n_components=1024, whiten=True)
pca = PCA(n_components=512, whiten=True)

# applying PCA to data
# must only fit on train data
history = pca.fit(X_train)

# checking explained variance
explained_var = pca.explained_variance_ratio_.sum()
print(f'PCA explained variance: {explained_var:.4f}')

### Logistic Regression

We then proceed to fit and evaluate a Logistic Regression. It's fairly easy and fast to fit it:

In [ ]:
# logistic regression and eval metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss

# instance of logistic regression
# C = Inverse of regularization strength
lr = LogisticRegression(C=1e-2, multi_class='multinomial', penalty='l2', max_iter=200)

# fitting to train
lr.fit(pca.transform(X_train), y_train)

In [ ]:
# evaluating
val_preds = lr.predict_proba(pca.transform(X_test))

# test metrics
print(f'Accuracy: {accuracy_score(y_test, np.argmax(val_preds, axis=1)):.3f}')
print(f'Log-loss: {log_loss(y_test, val_preds):.3f}')

#Original Results
*   Accuracy: 0.822
*   Log-loss: 0.774

#Xception Average Pooling
*   Accuracy: 0.823
*   Log-loss: 0.736

#Xception Max Pooling
*   Accuracy: 0.816
*   Log-loss: 0.797

#VGG19 Average Pooling
*   Accuracy: 0.706
*   Log-loss: 1.373

#VGG19 Max Pooling
*   Accuracy: 0.661
*   Log-loss: 1.486

#DenseNet201 Average Pooling
*   Accuracy: 0.870
*   Log-loss: 0.636

#DenseNet201 Max Pooling
*   Accuracy: 0.856
*   Log-loss: 0.664

#InceptionV3 Average Pooling
*   Accuracy: 0.819
*   Log-loss: 0.730

#InceptionV3 Max Pooling
*   Accuracy: 0.802
*   Log-loss: 0.769

### Predicting breed

In [ ]:
# define test image path
testImgPath = '/content/drive/MyDrive/CS465/project/greatWhitePyranees.jpeg'

In [ ]:
# features from zeca
features_zeca = extract_features([f'/content/drive/MyDrive/CS465/project/greatWhitePyranees.jpeg'], extractor)

# predictions for zeca
preds_zeca = lr.predict_proba(pca.transform(features_zeca))[0]
preds_zeca = pd.Series(preds_zeca, index=label_encoder.classes_)
preds_zeca.sort_values(ascending=False).to_frame().head(10)

### Explanations via embeddings and prototypes

One easy and effective method that I usually apply for explaining models is trying to transform them in a kNN (yeah, nearest neighbors!), as it outputs *hard examples* to support the model's decisions (or *prototypes*, as in the literature). How do we transform our Xception + PCA + Logistic Regession pipeline in a kNN, though? I'll show you two ways:

1. **Direct, naive way:** Just search for Zeca's neighbors in the `Xception` feature space after `PCA`
2. **Scale by Logistic Regression coefficients:** we apply a bit of **supervision** on the `Xception` + `PCA` embedding, scaling its features proportionally to the weights of the Logistic Regression.

Let us check how they perform. We start by importing `NNDescent`, a fast, efficient method to perform approximate nearest neighbor search:

In [ ]:
# nearest neighbors
from pynndescent import NNDescent

### Direct, naive embedding

Now, we search for Zeca's comparables in a naive way. It consists of creating an index on the `Xception` + `PCA` embedding, and then searching for zeca's neighbors in this index. The function `get_prototypes_report` takes care of that for us, and shows pictures and most frequent breeds for Zeca's neighbors:

In [ ]:
# creating NN index 
index_direct = NNDescent(pca.transform(X))

# running
get_prototypes_report(f'/content/drive/MyDrive/CS465/project/greatWhitePyranees.jpeg', index_direct, extractor, pca.transform, meta_df, features_df)

We start off OK on the first 10 dogs, but we still get neighbors that don't make much sense, like the `EntleBucher` or `Bouvier_des_Flandres`. Let us then improve that by applying a bit of **supervision** using the logistic regression's weights.

### Scale by Logistic Regression coefficients 

Let us perform a very simple modification to the embedding that our nearest neighbor method builds its index on. We use the fact that we can interpret the absolute value of the coefficients of the Logistic Regression as feature importances (as allowed by the whitening process), and scale the embedding features proportionally to these coefficients.

For instance, we can check that there are some features with nearly 10x more importance than others: 

In [ ]:
# checking feature importance
np.abs(lr.coef_).sum(axis=0)

So, when we scale the embedding this way, the Logistic Regression's most important features will have greater variance, and thus will have more weight when we search for Zeca's nearest neighbors:

In [ ]:
# function to 'supervise' embedding given coefficients of logreg
lr_coef_transform = lambda x: np.abs(lr.coef_).sum(axis=0) * pca.transform(x)

# creating NN index 
index_logistic = NNDescent(lr_coef_transform(X))

The results are much better:

In [ ]:
# running
get_prototypes_report(f'/content/drive/MyDrive/CS465/project/greatWhitePyranees.jpeg', index_logistic, extractor, lr_coef_transform, meta_df, features_df)

The prototypes agree a lot with the Logistic Regression results, and they're gonna be a solid argument for my family that the model works.

### Digging deeper: Why did the supervision work?

Why did simple scaling improve prototype quality by so much? My hypothesis is based on the curse on dimensionality. To check that, let us compare the 2D embedding generated by `UMAP` from the naive and scaled approaches.

We generate the embeddings with the following code:

In [ ]:
# UMAP for dimension reduction
from umap.umap_ import UMAP

# building embedding
umap_direct = UMAP()
embed_direct = umap_direct.fit_transform(pca.transform(X))

# predicting zeca
zeca_embed_direct = umap_direct.transform(pca.transform(features_zeca))

# building embedding
umap_logistic = UMAP()
embed_logistic = umap_logistic.fit_transform(lr_coef_transform(X))

# predicting zeca
zeca_embed_logistic = umap_logistic.transform(lr_coef_transform(features_zeca))

And plot them below:

In [ ]:
# opening figure
plt.figure(figsize=(16, 6), dpi=150)

# plotting 2D reduction of naive embedding
plt.subplot(1, 2, 1)
plot_embedding(embed_direct, zeca_embed_direct, 'Color is Breed: embedding directly from network', y)

# plotting 2D reduction of scaled embedding
plt.subplot(1, 2, 2)
plot_embedding(embed_logistic, zeca_embed_logistic, 'Color is Breed: embedding scaled by logistic regression weights', y)

Breed is color-coded in the plots. The naive embedding plot, in the left-hand side, shows reasonable structure, with some clear clusters of breeds clumping together. However, there's a "hubness" problem: there's a central clump of dog images where there's a lot of mix between breeds. I make my case that this is the curse of dimensionality at play: the 1024 features we get from the `Xception` and `PCA` are suited to a much more general problem of object idenfitication and are "too sparse" for our dog breed classification problem. Thus, we end up comparing pictures on features that don't make sense for our specific problem, making dogs that are different appear the same (and the contrary as well).

In the right-hand plot, built from the scaled embedding, we get much tighter, cleaner clusters, with no "hubness" at all. The scaling process acts like a "filter" letting we only compare pictures of dogs on the features that are important for our specific task of dog breed identification, as determined by our model. It's like **learning a distance** between our entities given our task.

For your amusement, I can also generate these plots using dog pictures. Here's what I call the **Dog Atlas**:

In [ ]:
# opening figure
fig, ax = plt.subplots(1, 2, figsize=(16, 6), dpi=150)

# plotting atlas
plot_dog_atlas(embed_direct, meta_df, 'Dog Atlas: naive embedding', ax[0])
plot_dog_atlas(embed_logistic, meta_df, 'Dog Atlas: scaled embedding', ax[1])

Cool! It's a fluffier way to see that the scaled embedding is better :)

### Final Remarks

Cool! We solved the mistery of Zeca's breed. We did not solve age, though, as we did not have the labels. I'll try on a next project.

Thank you very much for reading! Comments and feedbacks are appreciated :)

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()